# Downloading the Dataset

In [ ]:
!gdown --id 1JuawXQmYVkjpfL3H0blqjDrqw8V1lHrC

Downloading...
From: https://drive.google.com/uc?id=1JuawXQmYVkjpfL3H0blqjDrqw8V1lHrC
To: /home/akarofl/Desktop/Projects/Makkhi/IR A4/YASS_MAP/FIRE_Dataset_EN_2010.rar
133MB [00:25, 5.23MB/s]


# Extracting the Dataset

In [ ]:
!unrar x FIRE_Dataset_EN_2010.rar


UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from FIRE_Dataset_EN_2010.rar

Creating    FIRE_Dataset_EN_2010                                      OK
Extracting  FIRE_Dataset_EN_2010/en.qrels.76-125.2010.txt.gz             0  OK 
Extracting  FIRE_Dataset_EN_2010/en.topics.76-125.2010.txt               0  OK 
Extracting  FIRE_Dataset_EN_2010/English-Data.tgz                             1 1 1 2 2 2 3 3 3 4 4 4 5 5 5 6 6 6 6 7 7 7 8 8 8 9 9 9 99  OK 
All OK


In [ ]:
!tar -xvzf FIRE_Dataset_EN_2010/English-Data.tgz

ry_7925796.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070722_sports_story_8091125.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070416_sports_story_7652990.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070310_sports_story_7497024.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070702_sports_story_8003930.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070523_sports_story_7817201.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070421_sports_story_7676644.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070325_sports_story_7562923.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070702_sports_story_8003598.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070227_sports_story_7445769.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070128_sports_story_7316676.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070222_sports_story_7424674.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070727_sports_story_8112983.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070320_sports_story_7541322.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070119_sports_story_7281837.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070831_sports_story_8258236

In [ ]:
!gzip -d FIRE_Dataset_EN_2010/en.qrels.76-125.2010.txt.gz

# Importing necessary packages and libraries

In [ ]:
import os
import glob
from bs4 import BeautifulSoup
import lxml.html
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
import bisect
import math
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy

from sklearn.cluster import AgglomerativeClustering
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# Downloading the required NLTK packages

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/akarofl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/akarofl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/akarofl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/akarofl/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Preparing the data for use by YASS stemmer based on D2
In this section, some data preprocessing such as removing alphanumeric words, numbers, stopwords from words of FIRE Dataset has been performed. This is not necessary but has been done so that clustering takes less time.

In [ ]:
files = [] # List is storing file name and file content. File content is a list of words.
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~''' 
stop_words = set(stopwords.words('english'))

# Returns Part of Speech for lemmatization
def wordnet_pos(word):
    t = nltk.pos_tag([word])[0][1][0].lower()
    t_pos = {"j": postag.ADJ, "n": postag.NOUN, "v": postag.VERB, "r": postag.ADV}
    return t_pos.get(t, postag.NOUN)
i = 0
for subdir, dirs, filenames in os.walk('TELEGRAPH_UTF8/'):
    print('Progress: ',i)
    for file_obj in filenames:
      i = i+1
      filename = os.path.join(subdir, file_obj)
      # Extracting data and saving it to raw_text
      soup = BeautifulSoup(open(filename, "r").read())
      raw_text = soup.find('text').get_text()
      for sym in raw_text : 
        # Removing punctuation
        if sym in punc : 
          raw_text = raw_text.replace(sym, "")

      # Removing non-alphabetic text and Converting text to lower-case
      words = [word.lower() for word in raw_text.split() if word.isalpha()]
      # Removing stopwords
      words = [w for w in words if not w in stop_words]
      temp_file = []
      # Storing filename to list
      temp_file.append(str(filename))
      # Storing file data to list
      temp_file.append(words)
      # Appending data to 'files' list
      files.append(temp_file)
print("Total Documents: ", len(files))

temp_terms = []
i = 0
for curr_file in files : 
  for word in curr_file[1]:
    temp_terms.append(word)
  i = i + 1
  if(i%10000==0): print("Progress: ", i, " / ", len(files), "   Total Words: ", len(temp_terms))

# Converting set to sorted list
dataset = sorted(list(set(temp_terms)))
del temp_terms

Progress:  0
Progress:  0
Progress:  0
Progress:  675
Progress:  8290
Progress:  10565
Progress:  12283
Progress:  15911
Progress:  18148
Progress:  20103
Progress:  24403
Progress:  30447
Progress:  30447
Progress:  30480
Progress:  33434
Progress:  33462
Progress:  33513
Progress:  34176
Progress:  35090
Progress:  35174
Progress:  37289
Progress:  37415
Progress:  38579
Progress:  39515
Progress:  39534
Progress:  39585
Progress:  41800
Progress:  41857
Progress:  44791
Progress:  44791
Progress:  53896
Progress:  55889
Progress:  58543
Progress:  64793
Progress:  68121
Progress:  70923
Progress:  77355
Progress:  85890
Progress:  85890
Progress:  95433
Progress:  97786
Progress:  99917
Progress:  105419
Progress:  108496
Progress:  111250
Progress:  117397
Total Documents:  125586
Progress:  10000  /  125586    Total Words:  1897712
Progress:  20000  /  125586    Total Words:  4102081
Progress:  30000  /  125586    Total Words:  7227570
Progress:  40000  /  125586    Total Words:  

# Implementing YASS Stemmer using D2

In this section, a matrix made of terms from FIRE Dataset has been prepared for distance between each and every element of dataset. Then, using d2, clustering has been performed and a list of stems has been prepared. A dictionary of labels and words has also been prepared for finding the stems.

**The calculation of clusters has been performed in batched of 1000 for efficient and size-independent operations.**

In [ ]:
def distance2(seq1, seq2):
    size_x = len(seq1)
    size_y = len(seq2)

    max_size = max(size_x, size_y)
    min_size = min(size_x, size_y)

    m = min_size

    for i in range(max_size):
        if(i < min_size and seq1[i]!=seq2[i]):
            m = i
            break

    if m == 0: 
        return 10000

    d = 0
    for i in range(m, max_size):
        d = d + ((1/2**(i-m)))
    
    return d*(1/m)

print("Calculating clusters for full dataset...")

d2 = []
dataset_labels = []
total_clusters = 0
last_count = 0

print("Calculating D2 based distances...")
count = 0
for i in range(len(dataset)) : 
  d = []
  for j in range(last_count, min(last_count+500, len(dataset))) : 
    d.append(distance2(dataset[i], dataset[j]))
  d2.append(d)
  
  count = count + 1
  if count%500==0:
    last_count = count
    print("Progress: ", count, " / ", len(dataset))

    d_clustering = AgglomerativeClustering(linkage = 'average', distance_threshold = 0.8, n_clusters = None).fit(d2)
    labels = d_clustering.labels_
    num_clusters = d_clustering.n_clusters_
    
    new_labels = [x+total_clusters for x in labels]
    dataset_labels.extend(new_labels)

    total_clusters = total_clusters + num_clusters
    d2.clear()

td = {}

for i in range(len(dataset_labels)):
  if dataset_labels[i] in td.keys():
    td[dataset_labels[i]].append(i)
  else:
    td[dataset_labels[i]] = [i]

stems = []

count = 0
for i in range(total_clusters):
    min_size = 100
    stem = None
    for value in td[i]:
      if(len(dataset[value]) < min_size):
          min_size = len(dataset[value])
          stem = dataset[value]
    stems.append(stem)

    count += 1
    if count%20000==0:
      print("Progress: ", count, " / ", total_clusters)


print("Done.")

files_temp = []

for curr_file in files : 
  tempfile = []
  tempfile.append(curr_file[0])
  file_words = []
  for word in curr_file[1]:
    try: 
      stem_word = stems[dataset_labels[bisect.bisect_left(dataset, word)]]
      file_words.append(stem_word)
    except:
      continue
  tempfile.append(file_words)
  files_temp.append(tempfile)

print("Stemming Done.")

files = files_temp
del files_temp

Calculating clusters for full dataset...
Calculating D2 based distances...
Progress:  500  /  309812
Progress:  1000  /  309812
Progress:  1500  /  309812
Progress:  2000  /  309812
Progress:  2500  /  309812
Progress:  3000  /  309812
Progress:  3500  /  309812
Progress:  4000  /  309812
Progress:  4500  /  309812
Progress:  5000  /  309812
Progress:  5500  /  309812
Progress:  6000  /  309812
Progress:  6500  /  309812
Progress:  7000  /  309812
Progress:  7500  /  309812
Progress:  8000  /  309812
Progress:  8500  /  309812
Progress:  9000  /  309812
Progress:  9500  /  309812
Progress:  10000  /  309812
Progress:  10500  /  309812
Progress:  11000  /  309812
Progress:  11500  /  309812
Progress:  12000  /  309812
Progress:  12500  /  309812
Progress:  13000  /  309812
Progress:  13500  /  309812
Progress:  14000  /  309812
Progress:  14500  /  309812
Progress:  15000  /  309812
Progress:  15500  /  309812
Progress:  16000  /  309812
Progress:  16500  /  309812
Progress:  17000  /  

In [ ]:
print(files[0])

['TELEGRAPH_UTF8/2007_utf8/entertainment/1070814_entertainment_story_8192374.utf8', ['telegram', 'calcutta', 'enterta', 'thrill', 'conscious', 'oscar', 'nomine', 'ashvin', 'kumar', 'move', 'little', 'terroris', 'fulllife', 'feature', 'take', 'us', 'forest', 'ashvin', 'kumar', 'rsquod', 'forest', 'hope', 'oscars', 'even', 'thoug', 'ashvin', 'kumar', 'missa', 'best', 'live', 'action', 'short', 'film', 'academ', 'awara', 'little', 'terroris', 'weng', 'become', 'grounded', 'film', 'first', 'short', 'screened', 'commerce', 'theatre', 'fashio', 'design', 'ritu', 'kumara', 'son', 'reade', 'first', 'fulllife', 'feature', 'film', 'forest', 'shot', 'corbet', 'national', 'park', 'bandaras', 'national', 'park', 'thrill', 'environmend', 'conscious', 'starred', 'javed', 'jaaferi', 'nandaka', 'dev', 'sen', 'sali', 'ali', 'zaidi', 'taru', 'kumar', 'anku', 'vikal', 'internati', 'film', 'slated', 'septem', 'releas', 'india', 'reason', 'enough', 'chat', 'ashvin', 'little', 'terroris', 'acclaim', 'think',

# Extracting terms and calculating IDF for these terms

In [ ]:
# Storing unique words in a set
terms = []
idf = []

print("Collecting terms...")
# Adding words to terms set
i = 0
for curr_file in files : 
  for word in curr_file[1]:
    terms.append(word)
  i = i + 1
  if(i%10000==0): print("Progress: ", i, " / ", len(files), "   Total Words: ", len(terms))

# Converting set to sorted list
term_index = sorted(list(set(terms)))

print("Terms Collected. Total term count: ", len(term_index))
print("")


print("Calculating Document Frequency...")

# Initializing document frequency (df) matrix
df = [0 for x in range(len(term_index))]
i = 0
for curr_file in files : 
  temp_set = set(curr_file[1])
  for term in temp_set :
    # Searching for the index of term in term_index list using binary search (bisect)
    # Incrementing document frequency value for returned index in df list
    df[bisect.bisect_left(term_index, term)] = df[bisect.bisect_left(term_index, term)] + 1
  i = i + 1
  if(i%10000==0): print("Progress: ", i, " / ", len(files))
    
# Calculating IDF using df values
for doc in df : 
  idf.append(math.log10(len(files)/doc))

del terms

print("idf Calculated.")

Progress:  10000  /  125586    Total Words:  1897469
Progress:  20000  /  125586    Total Words:  4101715
Progress:  30000  /  125586    Total Words:  7226611
Progress:  40000  /  125586    Total Words:  9492733
Progress:  50000  /  125586    Total Words:  11629256
Progress:  60000  /  125586    Total Words:  13591177
Progress:  70000  /  125586    Total Words:  15911731
Progress:  80000  /  125586    Total Words:  18339486
Progress:  90000  /  125586    Total Words:  20618823
Progress:  100000  /  125586    Total Words:  22477404
Progress:  110000  /  125586    Total Words:  24735161
Progress:  120000  /  125586    Total Words:  27377518
Terms Collected. Total term count:  87385

Calculating Document Frequency...
Progress:  10000  /  125586
Progress:  20000  /  125586
Progress:  30000  /  125586
Progress:  40000  /  125586
Progress:  50000  /  125586
Progress:  60000  /  125586
Progress:  70000  /  125586
Progress:  80000  /  125586
Progress:  90000  /  125586
Progress:  100000  /  12

# Calculating Sparse TF-IDF matrix for documents

In [ ]:
sparse_tf_idf = []

j = 0
# Calculating TF-IDF vectors for each file
for doc in files :
  tempTF = {}
  # Incrementing count for each term
  for term in doc[1] : 
    index = bisect.bisect_left(term_index, term)
    if index in tempTF.keys():
        tempTF[index] += 1
    else:
        tempTF[index] = 1

  stf = []

  for i in tempTF:
        stf.append([i, tempTF[i]*idf[i]])     

  sparse_tf_idf.append(stf)

  j = j + 1
  if(j%10000==0): print("Progress: ", j, " / ", len(files))

print("tf-idf calculated for documents.")

Progress:  10000  /  125586
Progress:  20000  /  125586
Progress:  30000  /  125586
Progress:  40000  /  125586
Progress:  50000  /  125586
Progress:  60000  /  125586
Progress:  70000  /  125586
Progress:  80000  /  125586
Progress:  90000  /  125586
Progress:  100000  /  125586
Progress:  110000  /  125586
Progress:  120000  /  125586
tf-idf calculated for documents.


# Collecting words from queries and storing in proper format


In [ ]:
import xml.etree.ElementTree as ET
queryPath = "FIRE_Dataset_EN_2010/en.topics.76-125.2010.txt"
tree = ET.parse(queryPath)
root = tree.getroot()

queries = []

#Collecting Queries
for records in root.findall('top'):
    summary = records.find('num').text
    full = records.find('title').text

    words = [word.lower() for word in full.split() if word.isalpha()]
    # Removing stopwords
    words = [w for w in words if not w in stop_words]
    queries.append([summary, words])
print("Queries Collected")

Queries Collected


# Performing Stemming using YASS stemmer on Queries

In [ ]:
print("Stemming on Queries started...")

files_temp = []

for curr_file in queries : 
  tempfile = []
  tempfile.append(curr_file[0])
  file_words = []
  for word in curr_file[1]:
    try: 
      if(word in dataset):
        stem_word = stems[dataset_labels[bisect.bisect_left(dataset, word)]]
        file_words.append(stem_word)
      else:
        file_words.append(word)
    except:
      continue
  tempfile.append(file_words)
  files_temp.append(tempfile)

print("Stemming Done.")

queries = files_temp
del files_temp

print("Stemming on Queries Done.")

Stemming on Queries started...
Stemming Done.
Stemming on Queries Done.


# Creating TF-IDF for Queries

In [ ]:
print("Creating vectors for query terms...")

queries_tf_idf = []

for i in range(len(queries)):
    tempTF = [0] * len(term_index)
    word_tokens = queries[i][1]
       
    for w in word_tokens: 
        if w in term_index:
            tempTF[bisect.bisect_left(term_index, w)] += 1

    tempT = []

    for j in range(len(tempTF)):
        if(tempTF[j] > 0):
            tempT.append([j, tempTF[j]*idf[j]])

    queries_tf_idf.append(tempT)

print("Vectors generated for query terms.")

Creating vectors for query terms...
Vectors generated for query terms.


# Calculating magnitude for each vector in query and document corpus

In [ ]:
# Calculating magnitude for each vector in query and document corpus

qMag = []
dMag = []

for i in range(len(queries_tf_idf)):
    sum = 0
    for k in range(len(queries_tf_idf[i])):
        sum = sum + (queries_tf_idf[i][k][1] * queries_tf_idf[i][k][1] )
    qMag.append(math.sqrt(sum))

for i in range(len(sparse_tf_idf)):
    sum = 0
    for k in range(len(sparse_tf_idf[i])):
        sum = sum + (sparse_tf_idf[i][k][1] * sparse_tf_idf[i][k][1] )
    dMag.append(math.sqrt(sum))


# Calculating Rankings of Docs for each query

In [ ]:
print("Calculating Rankings of Docs for each query...")

r1 = []

for i in range(len(queries_tf_idf)):
    rTemp = []
    for j in range(len(sparse_tf_idf)):
        tSum = 0
        for k in range(len(queries_tf_idf[i])):
            for l in range(len(sparse_tf_idf[j])):
                if queries_tf_idf[i][k][0] == sparse_tf_idf[j][l][0]:
                    tSum += (queries_tf_idf[i][k][1] * sparse_tf_idf[j][l][1])
        if(tSum > 0):
            tSum = tSum / (qMag[i] * dMag[j])
            rTemp.append([files[j][0], tSum])
    rTemp.sort(key = lambda rTemp: rTemp[1], reverse=True) 
    r1.append(rTemp)

    print("Progress: ", i+1, " / ", len(queries_tf_idf))

print("Ranking Complete.")

Calculating Rankings of Docs for each query...
Progress:  1  /  50
Progress:  2  /  50
Progress:  3  /  50
Progress:  4  /  50
Progress:  5  /  50
Progress:  6  /  50
Progress:  7  /  50
Progress:  8  /  50
Progress:  9  /  50
Progress:  10  /  50
Progress:  11  /  50
Progress:  12  /  50
Progress:  13  /  50
Progress:  14  /  50
Progress:  15  /  50
Progress:  16  /  50
Progress:  17  /  50
Progress:  18  /  50
Progress:  19  /  50
Progress:  20  /  50
Progress:  21  /  50
Progress:  22  /  50
Progress:  23  /  50
Progress:  24  /  50
Progress:  25  /  50
Progress:  26  /  50
Progress:  27  /  50
Progress:  28  /  50
Progress:  29  /  50
Progress:  30  /  50
Progress:  31  /  50
Progress:  32  /  50
Progress:  33  /  50
Progress:  34  /  50
Progress:  35  /  50
Progress:  36  /  50
Progress:  37  /  50
Progress:  38  /  50
Progress:  39  /  50
Progress:  40  /  50
Progress:  41  /  50
Progress:  42  /  50
Progress:  43  /  50
Progress:  44  /  50
Progress:  45  /  50
Progress:  46  / 

# Extracting Data from qrels file

In [ ]:
file1 = open('FIRE_Dataset_EN_2010/en.qrels.76-125.2010.txt', 'r')
Lines = file1.readlines()

refDoc = []
for line in Lines:
    refDoc.append(line.split())

# Calculating MAP Score

In [ ]:
#List containing AP10 for each query
AP10 = []

for i in range(len(r1)):
    cAP = 0
    for j in range(min(10, len(r1[i]))):
        docName = r1[i][j][0].split('/')[3]
        queryNo = queries[i][0]

        #Search for this doc in reference file
        for k in range(len(refDoc)):
            if(refDoc[k][0] == queryNo and refDoc[k][2] == docName and refDoc[k][3] == '1'):
                cAP = cAP + 1
    AP10.append(cAP/10.0)

MAP10 = 0
for ap in AP10:
    MAP10 += ap

MAP10 /= 50

In [ ]:
print(MAP10)

0.22600000000000006
